In [16]:
import numpy as np
import pandas as pd
import sklearn
from glob import glob

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [17]:
logs = pd.read_csv('../data/mini/log_mini.csv')

In [18]:
logs.shape

(167880, 21)

In [19]:
print(logs.dtypes)

session_id                         object
session_position                    int64
session_length                      int64
track_id_clean                     object
skip_1                               bool
skip_2                               bool
skip_3                               bool
not_skipped                          bool
context_switch                      int64
no_pause_before_play                int64
short_pause_before_play             int64
long_pause_before_play              int64
hist_user_behavior_n_seekfwd        int64
hist_user_behavior_n_seekback       int64
hist_user_behavior_is_shuffle        bool
hour_of_day                         int64
date                               object
premium                              bool
context_type                       object
hist_user_behavior_reason_start    object
hist_user_behavior_reason_end      object
dtype: object


In [20]:
def categorical_to_dummies(df, categorical_cols):
    """ Create dummies (one hot encoding) for each categorical variables """
    dummies = pd.get_dummies(df[categorical_cols], prefix=categorical_cols)
    return df.drop(columns=categorical_cols).join(dummies)

In [21]:
# remove date for convenience (could encode this as well)
logs.drop(columns=['date'], inplace=True)

# Create dummies (one hot encoding) for each categorical variable in logs
categorical_cols = ['context_type', 'hist_user_behavior_reason_start', 'hist_user_behavior_reason_end']
logs = categorical_to_dummies(logs, categorical_cols)
print(logs.shape)

(167880, 39)


In [22]:
# Load in track features
track_features = pd.read_csv('../data/mini/tf_mini.csv').set_index('track_id')

In [23]:
print(track_features.dtypes)

duration                  float64
release_year                int64
us_popularity_estimate    float64
acousticness              float64
beat_strength             float64
bounciness                float64
danceability              float64
dyn_range_mean            float64
energy                    float64
flatness                  float64
instrumentalness          float64
key                         int64
liveness                  float64
loudness                  float64
mechanism                 float64
mode                       object
organism                  float64
speechiness               float64
tempo                     float64
time_signature              int64
valence                   float64
acoustic_vector_0         float64
acoustic_vector_1         float64
acoustic_vector_2         float64
acoustic_vector_3         float64
acoustic_vector_4         float64
acoustic_vector_5         float64
acoustic_vector_6         float64
acoustic_vector_7         float64
dtype: object


In [24]:
track_features.shape

(50704, 29)

In [25]:
# Create dummies (one hot encoding) for categorical variable in track_features
track_features = categorical_to_dummies(track_features, ['mode'])

In [26]:
track_features.head()

,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,mode_major,mode_minor
track_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
t_a540e552-16d4-42f8-a185-232bd650ea7d,109.706673,1950,99.975414,0.458040,0.519497,0.504949,0.399767,7.511880,0.817709,0.903753,3.254327e-06,0,0.132124,-11.238,0.386100,0.541606,0.079985,166.287003,4,0.935512,-0.033284,-0.411896,-0.028580,0.349438,0.832467,-0.213871,-0.299464,-0.675907,1,0
t_67965da0-132b-4b1e-8a69-0ef99b32287c,187.693329,1950,99.969430,0.916272,0.419223,0.545530,0.491235,9.098376,0.154258,1.037952,8.344854e-12,0,0.163281,-13.706,0.125000,0.895874,0.083877,95.261002,3,0.359675,0.145703,-0.850372,0.123860,0.746904,0.371803,-0.420558,-0.213120,-0.525795,1,0
t_0614ecd3-a7d5-40a1-816e-156d5872a467,160.839996,1951,99.602549,0.812884,0.425890,0.508280,0.491625,8.368670,0.358813,1.004268,2.927475e-10,0,0.090115,-10.522,0.200669,0.806136,0.038777,105.185997,4,0.726769,0.021720,-0.743634,0.333247,0.568447,0.411094,-0.187749,-0.387599,-0.433496,0,1
t_070a63a0-744a-434e-9913-a97b02926a29,175.399994,1951,99.665018,0.396854,0.400934,0.359990,0.552227,5.967346,0.514585,0.975138,3.981341e-11,0,0.360924,-11.032,0.427152,0.492772,0.038337,119.441002,4,0.859075,0.039143,-0.267555,-0.051825,0.106173,0.614825,-0.111419,-0.265953,-0.542753,1,0
t_d6990e17-9c31-4b01-8559-47d9ce476df1,369.600006,1951,99.991764,0.728831,0.371328,0.335115,0.483044,5.802681,0.721442,0.976866,2.541884e-10,0,0.189162,-6.836,0.281250,0.723808,0.032043,95.261002,4,0.562343,0.131931,-0.292523,-0.174819,-0.034422,0.717229,-0.016239,-0.392694,-0.455496,1,0


In [27]:
# Join track features and logs
data = logs.join(track_features, on='track_id_clean', how='left').drop(columns=['track_id_clean'])
data.head()

,session_id,session_position,session_length,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,premium,context_type_catalog,context_type_charts,context_type_editorial_playlist,context_type_personalized_playlist,context_type_radio,context_type_user_collection,hist_user_behavior_reason_start_appload,hist_user_behavior_reason_start_backbtn,hist_user_behavior_reason_start_clickrow,hist_user_behavior_reason_start_endplay,hist_user_behavior_reason_start_fwdbtn,hist_user_behavior_reason_start_playbtn,hist_user_behavior_reason_start_remote,hist_user_behavior_reason_start_trackdone,hist_user_behavior_reason_start_trackerror,hist_user_behavior_reason_end_backbtn,hist_user_behavior_reason_end_clickrow,hist_user_behavior_reason_end_endplay,hist_user_behavior_reason_end_fwdbtn,hist_user_behavior_reason_end_logout,hist_user_behavior_reason_end_remote,hist_user_behavior_reason_end_trackdone,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,mode_major,mode_minor
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,False,False,False,True,0,0,0,0,0,0,True,16,True,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,180.066666,2018,99.968133,0.015848,0.438551,0.473455,0.653119,7.660024,0.553465,1.035007,3.484468e-03,1,0.678553,-6.577,0.546784,0.320668,0.069717,134.024994,4,0.152255,-0.815775,0.386409,0.230160,0.028028,-0.333373,0.015452,-0.353590,0.205826,1,0
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,False,False,False,True,0,1,0,0,0,0,True,16,True,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,236.796371,2018,99.896728,0.061811,0.654804,0.735661,0.877393,11.308750,0.726828,1.025636,1.031315e-07,7,0.104322,-5.319,0.824766,0.131391,0.061158,130.037994,4,0.337152,-0.713646,0.363718,0.310315,-0.042222,-0.383164,0.066357,-0.365308,0.157920,0,1
2,0_00006f66-33e5-4de7-a324-2d18e439fc1e,3,20,False,False,False,True,0,1,0,0,0,0,True,16,True,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,231.266663,2018,99.999976,0.354116,0.532155,0.540411,0.679719,8.065802,0.563009,1.029465,2.659035e-08,10,0.135776,-5.843,0.774327,0.296923,0.045354,145.028000,4,0.373862,-0.742541,0.375599,0.252660,-0.049007,-0.299745,0.063341,-0.486689,0.181604,1,0
3,0_00006f66-33e5-4de7-a324-2d18e439fc1e,4,20,False,False,False,True,0,1,0,0,0,0,True,16,True,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,169.826675,2018,99.995038,0.769225,0.641756,0.729224,0.864881,11.287586,0.529484,0.993520,6.598388e-06,1,0.103722,-7.756,0.630996,0.603271,0.229936,111.982002,4,0.649420,-0.705116,0.317562,0.289141,-0.038920,-0.393358,0.092719,-0.364418,0.285603,1,0
4,0_00006f66-33e5-4de7-a324-2d18e439fc1e,5,20,False,False,False,True,0,1,0,0,0,0,True,16,True,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,210.545258,2018,99.998498,0.006602,0.732428,0.794881,0.857778,12.181586,0.650057,1.000571,2.066649e-06,8,0.120842,-4.919,0.759465,0.170148,0.240980,147.031006,4,0.652921,-0.868489,0.331280,0.210478,0.084740,-0.333287,-0.025706,-0.510350,0.182315,1,0


In [28]:
def split_sessions(data, perc_in=0.6):
    """ Split interactions into train and test sessions. """
    sessions = data['session_id'].unique()
    amt_in = int(perc_in * len(sessions))
    sessions_in = np.random.choice(sessions, amt_in, replace=False)
    sessions_out = np.array(list(set(sessions) - set(sessions_in)))
    indexed_data = data.set_index('session_id')
    data_in = indexed_data.loc[sessions_in]
    data_out = indexed_data.loc[sessions_out]
    return data_in, data_out

In [29]:
train, val = split_sessions(data, 0.6)
print(len(train), len(val))

100813 67067


In [30]:
def stack_sessions(df):
    """
    Turn matrix representation into vector by stacking the listen events together (as columns) 
    For example:
    session_id session_position feature1 feature2
    a          1                ~        ~
    a          2                ~        ~
    b          1                ~        ~
    b          2                ~        ~
    b          3                ~        ~
    
    Turns into:
    session_id 1_feature1 1_feature2 2_feature1 2_feature2 3_feature1 3_feature2
    a          ~          ~          ~          ~          Nan        Nan
    b          ~          ~          ~          ~          ~          ~
    """
    columns = list(df.columns)
    columns.remove('session_id')
    columns.remove('session_position')
    sessions = df.pivot(index='session_id', columns='session_position', values=columns)
    return sessions

In [31]:
# Columns to drop from the second part of the session, because this information is not available for prediction.
# Note that you could still use this information for learning embeddings etc.
drop_cols = list(logs.columns)
drop_cols.remove('session_id')
drop_cols.remove('session_position')
drop_cols.remove('track_id_clean')

In [32]:
def split_df(df):
    """
    Split df in data and labels part. First half of session is stacked and joined to each song in the second half of the session.
    Listening information is removed from second half, as it will not be available for prediction.
    """
    # Get sessions for SPOS<10 & reset index: old index added as col, new index added
    first = df.loc[df['session_position']*2 <= df['session_length']].reset_index().drop(columns=['session_length'])
    # Get sessions for SPOS>10 & reset index. 
    second = df.loc[df['session_position']*2 > df['session_length']].reset_index()
    # skip_2 is ground truth
    truth = second['skip_2']
    # Note: if you would print head; some sessions are smaller than 20 so SPOS can be < 10
    # print(second.head())
    # Note: track_id_clean has been dropped way before already
    second.drop(columns=drop_cols, inplace=True)
    first_stacked = stack_sessions(first)
    
    data = second.join(first_stacked, how='left', on='session_id')
    return data, truth

In [33]:
# Wrangle the train and validation sets in the right format.
train_data, train_labels = split_df(train)
val_data, val_labels = split_df(val)

/Users/jenselin/miniconda3/envs/Tensorflow/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jenselin/miniconda3/envs/Tensorflow/lib/python3.7/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [34]:
train_data.head()

,session_id,session_position,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,mode_major,mode_minor,"(skip_1, 1)","(skip_1, 2)","(skip_1, 3)","(skip_1, 4)","(skip_1, 5)","(skip_1, 6)","(skip_1, 7)","(skip_1, 8)","(skip_1, 9)","(skip_1, 10)","(skip_2, 1)","(skip_2, 2)","(skip_2, 3)","(skip_2, 4)","(skip_2, 5)","(skip_2, 6)","(skip_2, 7)","(skip_2, 8)","(skip_2, 9)","(skip_2, 10)","(skip_3, 1)","(skip_3, 2)","(skip_3, 3)","(skip_3, 4)","(skip_3, 5)","(skip_3, 6)","(skip_3, 7)","(skip_3, 8)","(skip_3, 9)","(skip_3, 10)","(not_skipped, 1)","(not_skipped, 2)","(not_skipped, 3)","(not_skipped, 4)","(not_skipped, 5)","(not_skipped, 6)","(not_skipped, 7)","(not_skipped, 8)","(not_skipped, 9)","(not_skipped, 10)","(context_switch, 1)","(context_switch, 2)","(context_switch, 3)","(context_switch, 4)","(context_switch, 5)","(context_switch, 6)","(context_switch, 7)","(context_switch, 8)","(context_switch, 9)","(context_switch, 10)","(no_pause_before_play, 1)","(no_pause_before_play, 2)","(no_pause_before_play, 3)","(no_pause_before_play, 4)","(no_pause_before_play, 5)","(no_pause_before_play, 6)","(no_pause_before_play, 7)","(no_pause_before_play, 8)","(no_pause_before_play, 9)","(no_pause_before_play, 10)","(short_pause_before_play, 1)","(short_pause_before_play, 2)","(short_pause_before_play, 3)","(short_pause_before_play, 4)","(short_pause_before_play, 5)","(short_pause_before_play, 6)","(short_pause_before_play, 7)","(short_pause_before_play, 8)","(short_pause_before_play, 9)","(short_pause_before_play, 10)","(long_pause_before_play, 1)","(long_pause_before_play, 2)","(long_pause_before_play, 3)","(long_pause_before_play, 4)","(long_pause_before_play, 5)","(long_pause_before_play, 6)","(long_pause_before_play, 7)","(long_pause_before_play, 8)","(long_pause_before_play, 9)","(long_pause_before_play, 10)","(hist_user_behavior_n_seekfwd, 1)","(hist_user_behavior_n_seekfwd, 2)","(hist_user_behavior_n_seekfwd, 3)","(hist_user_behavior_n_seekfwd, 4)","(hist_user_behavior_n_seekfwd, 5)","(hist_user_behavior_n_seekfwd, 6)","(hist_user_behavior_n_seekfwd, 7)","(hist_user_behavior_n_seekfwd, 8)","(hist_user_behavior_n_seekfwd, 9)","(hist_user_behavior_n_seekfwd, 10)","(hist_user_behavior_n_seekback, 1)","(hist_user_behavior_n_seekback, 2)","(hist_user_behavior_n_seekback, 3)","(hist_user_behavior_n_seekback, 4)","(hist_user_behavior_n_seekback, 5)","(hist_user_behavior_n_seekback, 6)","(hist_user_behavior_n_seekback, 7)","(hist_user_behavior_n_seekback, 8)","(hist_user_behavior_n_seekback, 9)","(hist_user_behavior_n_seekback, 10)","(hist_user_behavior_is_shuffle, 1)","(hist_user_behavior_is_shuffle, 2)","(hist_user_behavior_is_shuffle, 3)","(hist_user_behavior_is_shuffle, 4)","(hist_user_behavior_is_shuffle, 5)","(hist_user_behavior_is_shuffle, 6)","(hist_user_behavior_is_shuffle, 7)","(hist_user_behavior_is_shuffle, 8)","(hist_user_behavior_is_shuffle, 9)","(hist_user_behavior_is_shuffle, 10)","(hour_of_day, 1)","(hour_of_day, 2)","(hour_of_day, 3)","(hour_of_day, 4)","(hour_of_day, 5)","(hour_of_day, 6)","(hour_of_day, 7)","(hour_of_day, 8)","(hour_of_day, 9)","(hour_of_day, 10)","(premium, 1)","(premium, 2)","(premium, 3)","(premium, 4)","(premium, 5)","(premium, 6)","(premium, 7)","(premium, 8)","(premium, 9)","(premium, 10)","(context_type_catalog, 1)","(context_type_catalog, 2)","(context_type_catalog, 3)","(context_type_catalog, 4)","(context_type_catalog, 5)","(context_type_catalog, 6)","(context_type_catalog, 7)","(context_type_catalog, 8)","(context_type_catalog, 9)","(context_type_catalog, 10)","(context_type_charts, 1)","(context_type_charts, 2)","(context_type_charts, 3)","(context_type_charts, 4)","(context_type_ch

In [35]:
val_data.head()

,session_id,session_position,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,flatness,instrumentalness,key,liveness,loudness,mechanism,organism,speechiness,tempo,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7,mode_major,mode_minor,"(skip_1, 1)","(skip_1, 2)","(skip_1, 3)","(skip_1, 4)","(skip_1, 5)","(skip_1, 6)","(skip_1, 7)","(skip_1, 8)","(skip_1, 9)","(skip_1, 10)","(skip_2, 1)","(skip_2, 2)","(skip_2, 3)","(skip_2, 4)","(skip_2, 5)","(skip_2, 6)","(skip_2, 7)","(skip_2, 8)","(skip_2, 9)","(skip_2, 10)","(skip_3, 1)","(skip_3, 2)","(skip_3, 3)","(skip_3, 4)","(skip_3, 5)","(skip_3, 6)","(skip_3, 7)","(skip_3, 8)","(skip_3, 9)","(skip_3, 10)","(not_skipped, 1)","(not_skipped, 2)","(not_skipped, 3)","(not_skipped, 4)","(not_skipped, 5)","(not_skipped, 6)","(not_skipped, 7)","(not_skipped, 8)","(not_skipped, 9)","(not_skipped, 10)","(context_switch, 1)","(context_switch, 2)","(context_switch, 3)","(context_switch, 4)","(context_switch, 5)","(context_switch, 6)","(context_switch, 7)","(context_switch, 8)","(context_switch, 9)","(context_switch, 10)","(no_pause_before_play, 1)","(no_pause_before_play, 2)","(no_pause_before_play, 3)","(no_pause_before_play, 4)","(no_pause_before_play, 5)","(no_pause_before_play, 6)","(no_pause_before_play, 7)","(no_pause_before_play, 8)","(no_pause_before_play, 9)","(no_pause_before_play, 10)","(short_pause_before_play, 1)","(short_pause_before_play, 2)","(short_pause_before_play, 3)","(short_pause_before_play, 4)","(short_pause_before_play, 5)","(short_pause_before_play, 6)","(short_pause_before_play, 7)","(short_pause_before_play, 8)","(short_pause_before_play, 9)","(short_pause_before_play, 10)","(long_pause_before_play, 1)","(long_pause_before_play, 2)","(long_pause_before_play, 3)","(long_pause_before_play, 4)","(long_pause_before_play, 5)","(long_pause_before_play, 6)","(long_pause_before_play, 7)","(long_pause_before_play, 8)","(long_pause_before_play, 9)","(long_pause_before_play, 10)","(hist_user_behavior_n_seekfwd, 1)","(hist_user_behavior_n_seekfwd, 2)","(hist_user_behavior_n_seekfwd, 3)","(hist_user_behavior_n_seekfwd, 4)","(hist_user_behavior_n_seekfwd, 5)","(hist_user_behavior_n_seekfwd, 6)","(hist_user_behavior_n_seekfwd, 7)","(hist_user_behavior_n_seekfwd, 8)","(hist_user_behavior_n_seekfwd, 9)","(hist_user_behavior_n_seekfwd, 10)","(hist_user_behavior_n_seekback, 1)","(hist_user_behavior_n_seekback, 2)","(hist_user_behavior_n_seekback, 3)","(hist_user_behavior_n_seekback, 4)","(hist_user_behavior_n_seekback, 5)","(hist_user_behavior_n_seekback, 6)","(hist_user_behavior_n_seekback, 7)","(hist_user_behavior_n_seekback, 8)","(hist_user_behavior_n_seekback, 9)","(hist_user_behavior_n_seekback, 10)","(hist_user_behavior_is_shuffle, 1)","(hist_user_behavior_is_shuffle, 2)","(hist_user_behavior_is_shuffle, 3)","(hist_user_behavior_is_shuffle, 4)","(hist_user_behavior_is_shuffle, 5)","(hist_user_behavior_is_shuffle, 6)","(hist_user_behavior_is_shuffle, 7)","(hist_user_behavior_is_shuffle, 8)","(hist_user_behavior_is_shuffle, 9)","(hist_user_behavior_is_shuffle, 10)","(hour_of_day, 1)","(hour_of_day, 2)","(hour_of_day, 3)","(hour_of_day, 4)","(hour_of_day, 5)","(hour_of_day, 6)","(hour_of_day, 7)","(hour_of_day, 8)","(hour_of_day, 9)","(hour_of_day, 10)","(premium, 1)","(premium, 2)","(premium, 3)","(premium, 4)","(premium, 5)","(premium, 6)","(premium, 7)","(premium, 8)","(premium, 9)","(premium, 10)","(context_type_catalog, 1)","(context_type_catalog, 2)","(context_type_catalog, 3)","(context_type_catalog, 4)","(context_type_catalog, 5)","(context_type_catalog, 6)","(context_type_catalog, 7)","(context_type_catalog, 8)","(context_type_catalog, 9)","(context_type_catalog, 10)","(context_type_charts, 1)","(context_type_charts, 2)","(context_type_charts, 3)","(context_type_charts, 4)","(context_type_ch

In [36]:
val_data.columns

Index([            'session_id',       'session_position',
                     'duration',           'release_year',
       'us_popularity_estimate',           'acousticness',
                'beat_strength',             'bounciness',
                 'danceability',         'dyn_range_mean',
       ...
              ('mode_minor', 1),        ('mode_minor', 2),
              ('mode_minor', 3),        ('mode_minor', 4),
              ('mode_minor', 5),        ('mode_minor', 6),
              ('mode_minor', 7),        ('mode_minor', 8),
              ('mode_minor', 9),       ('mode_minor', 10)],
      dtype='object', length=682)

In [37]:
%%time
from sklearn.impute import SimpleImputer

# Fill in NaN values with mean of column. This choice *should* depend on your decision of classifier and feature encoding. Not every approach has the same assumptions about the data.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
train_data_imp = imp.fit_transform(train_data.drop(columns=['session_id']))
val_data_imp = imp.transform(val_data.drop(columns=['session_id']))

CPU times: user 7.55 s, sys: 2.38 s, total: 9.94 s
Wall time: 10.5 s


In [38]:
# Print samples to learn from and features per sample.
print("train matrix shape:", train_data.shape)
print("labels shape:", train_labels.shape)

train matrix shape: (51123, 682)
labels shape: (51123,)


In [39]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Train a classifier (default parameters are not optimal here!)
model = LogisticRegression(max_iter=500)
model.fit(train_data_imp, train_labels.values)

CPU times: user 1min 2s, sys: 432 ms, total: 1min 2s
Wall time: 16.1 s


/Users/jenselin/miniconda3/envs/Tensorflow/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(max_iter=500)

In [40]:
%%time
# Predict validation data
predictions = model.predict(val_data_imp)

CPU times: user 74.5 ms, sys: 3.22 ms, total: 77.8 ms
Wall time: 25.7 ms


In [41]:
# Convert from flattened format back to session based format (for calculation of MAA metric).
prediction_df = val_data[['session_id']]
prediction_df.loc[:, 'prediction'] = predictions
predictions_list = prediction_df.groupby('session_id')['prediction'].apply(list)

labels_df = val_data[['session_id']]
labels_df.loc[:, 'truth'] = val_labels
truth_list = labels_df.groupby('session_id')['truth'].apply(list)

/Users/jenselin/miniconda3/envs/Tensorflow/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jenselin/miniconda3/envs/Tensorflow/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [42]:
def evaluate(submission, groundtruth):
    """ Calculate metrics for prediction and ground thruth lists (source: starter kit) """
    ap_sum = 0.0
    first_pred_acc_sum = 0.0
    counter = 0
    for sub, tru in zip(submission, groundtruth):
        if len(sub) != len(tru):
            raise Exception('Line {} should contain {} predictions, but instead contains '
                            '{}'.format(counter+1,len(tru),len(sub)))
        try:
            ap_sum += ave_pre(sub,tru)
        except ValueError as e:
            raise ValueError('Invalid prediction in line {}, should be 0 or 1'.format(counter))
        first_pred_acc_sum += sub[0] == tru[0]
        counter+=1
    ap = ap_sum/counter
    first_pred_acc = first_pred_acc_sum/counter
    return ap,first_pred_acc

In [43]:
def ave_pre(submission,groundtruth):
    """ Calculate average accuracy (which is the same as average precision in this context) """
    s = 0.0
    t = 0.0
    c = 1.0
    for x, y in zip(submission, groundtruth):
        if x != 0 and x != 1:
            raise ValueError()
        if x==y:
            s += 1.0
            t += s / c
        c += 1
    return t/len(groundtruth)

In [44]:
ap,first_pred_acc = evaluate(predictions_list, truth_list)

In [45]:
print('average precision: {}'.format(ap))
print('first prediction accuracy: {}'.format(first_pred_acc))

average precision: 0.5194698864244776
first prediction accuracy: 0.68025
